<a href="https://colab.research.google.com/github/falconlee236/DeepLearningFrom_Scratch/blob/main/Book_2/ch04/Chapter_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 4 word2vec speed improvement

**4.1 word2vec improvement**

*4.1.2 Embedding layer implementation*

In [ ]:
import numpy as np
W = np.arange(21).reshape(7, 3)
W

In [ ]:
W[2]

In [ ]:
W[5]

In [ ]:
idx = np.array([1, 0, 3, 0])
W[idx]

In [ ]:
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
    
    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out
    
    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0

        np.add.at(dW, self.idx, dout)

        return None

**4.2 word2vec improvement 2**

*4.2.4 from multi-class classification to binary classification(implementation)*

In [ ]:
class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grad
        self.cache = None
    
    def forward(self, h, idx):
        target_W = self.embed.forwad(idx)
        out = np.sum(target_W * h, axis=1)

        self.cache = (h, target_W)
        return out

    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1)
        
        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh

*4.2.6 Sampling technique of Negative Sampling *

In [ ]:
import numpy as np

# random sampling number between 0 to 9 
np.random.choice(10)

In [ ]:
np.random.choice(10)

In [ ]:
# random one number sampling from words list
words = ['you', 'say', 'goodbye', 'I', 'hello', '.']
np.random.choice(words)

In [ ]:
# random 5 number sampling (multiple selection exist)
np.random.choice(words, size=5)

In [ ]:
# random 5 number sampling (multiple selection not exist)
np.random.choice(words, size=5, replace=False)

In [ ]:
# sampling about probability distribution
p = [0.5, 0.1, 0.05, 0.2, 0.05, 0.1]
np.random.choice(words, p=p)

In [ ]:
p = [0.7, 0.29, 0.01]
new_p = np.power(p, 0.75)
new_p /= sum(new_p)
print(new_p)

In [ ]:
%cd drive/MyDrive/DeepLearningFrom_Scratch/Book2/ch04

In [ ]:
from negative_sampling_layer import UnigramSampler

corpus = np.array([0, 1, 2, 3, 4, 1, 2, 3])
power = 0.75
sample_size = 2

sampler = UnigramSampler(corpus, power, sample_size)
target = np.array([1, 3, 0])
negative_sample = sampler.get_negative_sample(target)
print(negative_sample)

*4.2.7 Negative Sampling implementation*

In [ ]:
class NegativeSamplingLoss:
    def __init__(self, W, corpus, power=0.75, sample_size=5):
        self.sample_size = sample_size
        self.sampler = UnigramSampler(corpus, power, sample_size)
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size + 1)]
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size + 1)]
        self.params, self,grads = [], []
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads
    
    def forward(self, h, target):
        batch_size = target.shape[0]
        negative_sample = self.sampler.get_negative_sample(target)

        # positive example forwarding
        score = self.embed_dot_layers[0].forward(h, target)
        correct_label = np.ones(batch_size, dype=np.int32)
        loss = self.loss_layers[0].forward(score, correct_label)

        # negative example forwarding
        negative_label = np.zeros(batch_size, dtype=np.int32)
        for i in range(self.sample_size):
            negative_target = negative_sample[:, i]
            score = self.embed_dot_layers[i + 1].forward(h, negative_target)
            loss += self.loss_layers[i + 1].forward(score, negative_label)

        return loss
    
    def backward(self, dout=1):
        dh = 0
        for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
            dscore = l0.backward(dout)
            dh += l1.backward(dscore)
        
        return dh